# Introduction to PINNs
### The static, non-linear Euler-Bernoulli beam

---

Lecture: "Physics-augmented machine learning" @ Cyber-Physical Simulation, TU Darmstadt

Lecturer: Prof. Oliver Weeger

Author: Jasper O. Schommartz

---

#### In this notebook, you will...

* learn how to calibrate PINNs of multidimensional solutions.
* learn how to use mixed methods for improving PINN calibration.
* experience the benefit of mixed methods for non-linear problems.
* modify boundary conditions for PINN calibration.


In [ ]:
# Run this cell is you are using Google Colab
!git clone https://github.com/CPSHub/LecturePhysicsAwareML.git
%cd LecturePhysicsAwareML/PINNs
%pip install -e .

In [ ]:
# Run this cell, if you are working locally. Remember to create a virtual 
# environment and activate it before running this cell.
%cd ..
%pip install -e .

## 1. Theory

### 1.1 Non-linear Euler-Bernoulli Beam

The quasi-static geometrically non-linear Euler-Bernoulli beam is described by the following set of equations:

$$
\begin{align}
- N' = f(x) \tag{1}\\
-(N w')' - M'' = q(x) \tag{2}
\end{align}
$$

with

$$
\begin{align}
N = EA (u' + \frac{1}{2} w'^2) \tag{3} \\
M = - EI w'' \tag{4} \\
Q = M' + N w' \tag{5}
\end{align}
$$

Here, $x$ denoted the position on the beam, and $u$ and $w$ denote horizontal and vertial displacements, respectively. The internal normal force, bending moment, and transversal force are denoted by $N$, $M$ and $Q$ respectively.

For the case of a cantilever beam, for which there is an analytical solution, the boundary conditions read

$$
u(0) = 0, \quad w(0) = 0, \quad w'(0) = 0 \tag{6}
$$

and

$$
N(L) = 0, \quad M(L) = 0, \quad Q(L) = F \tag{7}
$$

### 1.2 Naive PINN Model

<img src="../img/pinn_nlebb.png" alt="PINN NLEBB" width=600>

As in the linear case, the naive PINN model for the coupled Equations (1) and (2) with boundary conditions (6) and (7) consists of a simple FFNN, which maps directly from the collocation point $x$ to the solutions $u$ and $w$. To compute the resisual-based loss term, up to fourth-order derivatives of the FFNN are computed for the residuals.

**Note**, since the present system consists of two coupled equations, the residual loss $\mathcal{L}_{\text{res}} = \mathcal{L}_{\text{res,1}} + \mathcal{L}_{\text{res,2}}$ consists of one term for each residual. Likewise, there is one residual term for every boundary condition.

### 1.3 Mixed PINN Model

<img src="../img/pinn_nlebb_mixed.png" alt="PINN NLEBB" width=600>

In the mixed formulation, the aim is to reduce the order of differentiation required during the training process with the goal of improving convergence [[1][Faroughi2023]]. This is achieved by reformulating Equations (1) and (2) into a larger system of equations with at most one order of differentiation. The following coupled system of equations is formulated:

$$
\begin{align*}
-N &= f(x) \\
- Q' &= q(x) \\
\theta &= w' \\
N &= EA(u' + \frac{1}{2} \theta^2) \\
M &= -EI \theta' \\
Q &= M' + N \theta \\
\end{align*}
$$

**Note** that in contrast to Section 1.1, $u$, $w$, $\theta$, $N$, $M$, and $Q$ are now treated as separate solution functions, which shall be approximated by separate neural networks during PINN calibration.

Using the mixed formulation, the boundary condition for the cantilever beam now read

$$
u(0) = 0, \quad w(0) = 0, \quad \theta(0) = 0
$$

and

$$
N(L) = 0, \quad M(L) = 0, \quad Q(L) = F
$$

Here, the boundary conditions now act directly on the variables and nolonger on derivatives.

The resulting PINN structure for the mixed formulation now consists of six separate FFNNs for each solution variable (One could also use one FFNN with six outputs). Hence, the number of residual terms in the loss function increases from two to 6 as well. The number of boundary condition residual terms remains unchanged.

[Faroughi2023]: https://doi.org/10.1007/s00707-023-03676-2

## 2. PINN Calibration

### 2.1 Definition of Boundary Conditions

You can use this section to define arbitrary boundary conditions on the NLEBB.

In [ ]:
import jax.numpy as jnp
from paml_pinns.nlebb import Config, get_data_decorator


@get_data_decorator
def get_data(config: Config):
    if config.bc_case == 0:
        u_bc_coords = jnp.array([0.0])
        u_bc_values = jnp.array([0.0])
        w_bc_coords = jnp.array([0.0])
        w_bc_values = jnp.array([0.0])
        w_x_bc_coords = jnp.array([0.0])
        w_x_bc_values = jnp.array([0.0])
        N_bc_coords = jnp.array([config.L])
        N_bc_values = jnp.array([0.0])
        M_bc_coords = jnp.array([config.L])
        M_bc_values = jnp.array([0.0])
        Q_bc_coords = jnp.array([config.L])
        Q_bc_values = jnp.array([config.F])
    else:
        NotImplementedError(
            "Data generation is not implemented for these boundary conditions."
        )

    bc = {
        "u_bc_coords": u_bc_coords,
        "u_bc_values": u_bc_values,
        "w_bc_coords": w_bc_coords,
        "w_bc_values": w_bc_values,
        "w_x_bc_coords": w_x_bc_coords,
        "w_x_bc_values": w_x_bc_values,
        "N_bc_coords": N_bc_coords,
        "N_bc_values": N_bc_values,
        "M_bc_coords": M_bc_coords,
        "M_bc_values": M_bc_values,
        "Q_bc_coords": Q_bc_coords,
        "Q_bc_values": Q_bc_values,
    }
    return bc


### 2.2 Configuration of Beam Properties

In this section, the properties of the beam can be defined for the boundary condition cases above. Note that line loads are considered to be constant.

In [ ]:
from paml_pinns.nlebb import get_config_decorator
from typing import Tuple


@get_config_decorator
def get_config(bc_case: int) -> Tuple[float, ...]:
    if bc_case == 0:
        E = 1.0
        A = 1.0
        II = 1.0
        L = 1.0
        F = 1.0
        f = 0.0
        q = 0.0
    else:
        raise NotImplementedError(
            "No configuration implemented for these boundary conditions."
        )

    return E, A, II, L, F, f, q

### 2.3 Data Generation and Model Creation

Here, we generate input and solution data for the LEBB. This is only possible, since Equation (1) can be solved analytically for the boundary conditions of a cantilever beam. However, **note** that solution data is not required for PINN calibration. Next, and instance of the PINN model is create using a pseudo-random seed.

In [ ]:
import jax
from paml_pinns.nlebb import create_pinn


config = get_config(bc_case=0)
x, y, bc, params = get_data(config)

key = jax.random.PRNGKey(1234)
model = create_pinn(params, bc, key=key, mixed_formulation=False)
weights = {"u": 1.0, "w": 1.0, "w_x": 1.0, "N": 1.0, "M": 1.0, "Q": 1.0}
params

### 2.4 Model Creation and Evaluation

The model is calibrated and evaluated. **Note**, that the solution data `y` is not passed to the `train` function. Hence, the PINN is trained exclusively using residuals.

In [ ]:
from paml_pinns import train
from paml_pinns.nlebb import evaluate

model = train(model, x, weights, steps=50_000, batch_size=128)
evaluate(model, x, y)

## 3. Tasks

### Task 1: Effect of Mixed Formulations

For `bc_case=0`, ...

a) Set `mixed_formulation=False`. Can the model approximate the solution?

b) Now set `mixed_formulation=True`, while keeping all other parameters the same. Can the model approximate the solution now? How can the observed differences be explained?

c) Next select your preferred architecture and increase the value of `E`. What do you observe? How could the effects be mitigated?

### Task 2: Discussion

Based on your experience from the the PINN task, collect some advantages and disadvantages of PINNs over traditional numerical methods like FEM. When would you prefer to use PINNs, when would you obt for FEM (or equivalent methods)? Discuss your thoughts with a partner.

### Task 3 (Optional): Implementation of Boundary Conditions

Calibrate the PINN to different boundary conditions by modifying the `get_data`
function and the `paml_pinn.nlebb.cases` module. Can you find an analytical soulution to validate your implementation?